# Classificação de manifestações de Ouvidoria usando Processamento de Linguagem Natural

**PONTIFÍCIA UNIVERSIDADE CATÓLICA DE MINAS GERAIS**

_Pós-graduação Lato Sensu em Inteligência Artificial e Aprendizado de Máquina_

Trabalho de Conclusão de Curso apresentado ao Curso de Especialização em Inteligência Artificial e Aprendizado de Máquina, como requisito parcial à obtenção do título de Especialista.

Aluno: Felipe Schiavon de Oliveira

Data: Abril/2022


# Configurando ambiente para base de dados

In [ ]:
# Importando pacotes
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import unidecode
from wordcloud import WordCloud, ImageColorGenerator

In [ ]:
# Desabilitando mensagens de alerta
import warnings
warnings.filterwarnings('ignore', '.*do not.*', )
warnings.warn('Do not show this message')

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
# Definindo padrão para mostrar todas as linhas e colunas no pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Definindo padrão para mostrar conteúdo total nas células do pandas
pd.set_option("display.max_colwidth", None)

# Ajuste de base de dados

## Leitura e preparação da base

In [ ]:
# Leitura do dataset
df_tj = pd.read_csv('./data/raw/sisouv_manifestacoes.csv', sep=';')
df_tj.head()

In [ ]:
# Contagem de registros do Dataframe
df_tj['id'].count()

In [ ]:
# Contagem de registros por tipo de manifestação
df_tj['tipoDeManifestacao'].value_counts()

In [ ]:
# Relatos com a string "Não informado"
len(df_tj.loc[df_tj['relato'] == 'Não Informado', :])

In [ ]:
# Total de relatos sem a string "Não informado"
len(df_tj) - len(df_tj.loc[df_tj['relato'] == 'Não Informado', :])

In [ ]:
# Contagem de registros por tipo de manifestação, excluindo relato "Não informado" 
df_tj.loc[df_tj['relato'] != 'Não Informado', 'tipoDeManifestacao'].value_counts()

In [ ]:
# Verificar relato em que tipo de manifestação é INFORMAÇÃO e o relato não é "Não Informado"
df_tj.loc[(df_tj['relato'] != 'Não Informado') & (df_tj['tipoDeManifestacao'] == 'INFORMACAO'), 'relato']

In [ ]:
# Criar Dataframe excluindo registros em que tipo de manifestação é INFORMACAO ou com relato "Não Informado"
df_relatos = df_tj.loc[(df_tj['relato'] != 'Não Informado') & (df_tj['tipoDeManifestacao'] != 'INFORMACAO'), :]
df_relatos.head()

In [ ]:
# Contagem de valores do novo Dataframe de acordo com tipo de manifestação
df_relatos['tipoDeManifestacao'].value_counts()

In [ ]:
# Contagem de registros do novo Dataframe
df_relatos['id'].count()

### Criando Treemap

In [ ]:
# Obtendo quantidade de regisros por tipo de manifestação
df_tj_graf1 = df_relatos['tipoDeManifestacao'].value_counts()
df_tj_graf1

In [ ]:
type(df_tj_graf1)

In [ ]:
# Transformando dados em data frame 
df_tj_graf1 = pd.DataFrame(df_tj_graf1.reset_index())
df_tj_graf1.columns = ['tipoDeManifestacao', 'qtdManifestacao']
df_tj_graf1

In [ ]:
# Criando gráfico treemap
import plotly.express as px
import plotly.graph_objects as go

fig = px.treemap(df_tj_graf1, path=['tipoDeManifestacao'],
                 values='qtdManifestacao')

fig.add_trace(go.Treemap(
    labels = df_tj_graf1['tipoDeManifestacao'],
    values =  df_tj_graf1['qtdManifestacao']))

fig.data[0].textinfo = 'label+text+value+percent root'
fig.layout.hovermode = False
fig.show()

### Criando Gráfico de Colunas

In [ ]:
# Criando data frame com tipo de manifestação e ano de cadastro
df_tj_graf2 = df_tj[['dataCadastro', 'tipoDeManifestacao']].copy()
df_tj_graf2['ano'] = pd.DatetimeIndex(df_tj_graf2['dataCadastro']).year
df_tj_graf2.head()

In [ ]:
# Obtendo quantidade de registros por ano de cadastro
qtd_manifestacao = pd.DataFrame(df_tj_graf2.groupby(['ano']).size().reset_index())
qtd_manifestacao.columns = ['ano', 'qtd']
qtd_manifestacao

In [ ]:
# Criando gráfico de coluna
plt.figure(figsize=(10,5))

ax = sns.barplot(x="ano", y="qtd", data=qtd_manifestacao, color="c")
ax.set(xlabel='Ano', ylabel='Quantidade');

### Análise de relatos

In [ ]:
# Verificando conteúdo de relatos
df_relatos.head(50)

## Tratamento da base

### Remoção HTML

In [ ]:
# Criando coluna com limpeza de HTML da coluna "relato" usando Beautiful Soap
df_relatos.loc[:,'relato_limpo'] = df_relatos['relato'].apply(lambda relato: bs(relato).text)

df_relatos.head()

In [ ]:
df_relatos.loc[:1000, 'relato_limpo']

### Remoção quebras de linha

In [ ]:
# Fazendo limpeza de quebra de linha e espaço sem quebra
df_relatos['relato_limpo'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r", "\xa0"],
                                   value=[" "," "," "],
                                   regex=True,
                                   inplace=True)

df_relatos.head()

### Remoção de texto padrão de e-mail

In [ ]:
# Removendo texto padrão de envio de e-mail da coluna "relato_limpo"
df_relatos['relato_limpo'] = df_relatos['relato_limpo'].str.replace(r'^.*?Assunto:','',
                                                                    regex=True)

df_relatos.head()

### Remoção de sinais de pontuação

In [ ]:
# Removendo sinais de pontuação
df_relatos["relato_limpo"] = df_relatos['relato_limpo'].str.replace('[^\w\s]+','', regex=True)
df_relatos.head()

### Remoção de excesso de espaços em branco

In [ ]:
# Removendo excesso de espaços em branco 
df_relatos['relato_limpo'] = df_relatos['relato_limpo'].str.replace('\s{2,100}', ' ', regex=True)
df_relatos.iloc[:5, -1]

In [ ]:
# Verificando os primeiros 20 registros do tipo DUVIDA
df_relatos.loc[df_relatos['tipoDeManifestacao'] == 'DUVIDA'].head(20)

### Transformação para lowercase

In [ ]:
# Transformando texto para lowercase
df_relatos['relato_limpo'] = df_relatos['relato_limpo'].str.lower()

In [ ]:
# Verificando 10 últimos registros da coluna de relato limpa
df_relatos.iloc[0:10, -1]

### Remoção de stopwords

In [ ]:
# Baixando stopwords
nltk.download('stopwords')

In [ ]:
# Carregando stopwords em português
from nltk.corpus import stopwords

stop_words = stopwords.words('portuguese')
stop_words[:10]

In [ ]:
# Adicionando novas stopwords
stop_words_adicionais = ['todo', 'todos', 'pois', 'pra', 'nao']
stop_words.extend(stop_words_adicionais)

In [ ]:
# Verificando stopwords adicionados
stop_words[-5:]

In [ ]:
# Aplicando stopwords à coluna relato_limpo
df_relatos['relato_limpo'] = df_relatos['relato_limpo'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# Verificando alguns exemplos
df_relatos.iloc[:5, -1]

### Substituindo caracteres com acento

In [ ]:
# Aplicando a remoção de acentos
df_relatos['relato_limpo'] = df_relatos['relato_limpo'].apply(lambda relato: unidecode.unidecode(relato))

# Verificando alguns exemplos
df_relatos.iloc[0:5, -1]

### Mascarando números

In [ ]:
# Inserir código
df_relatos['relato_limpo'] = df_relatos['relato_limpo'].str.replace(r"\d", "", regex=True)

# Verificando alguns exemplos
df_relatos.iloc[:5, -1]

### Remoção de caracteres únicos

In [ ]:
# Definindo caracteres a serem removidos
car_unicos = ['h', 'n', 'b', 'q', 'i', 'ii']

# Aplicando caracteres únicos à coluna relato_limpo
df_relatos['relato_limpo'] = df_relatos['relato_limpo'].apply(lambda x: ' '.join([word for word in x.split() if word not in (car_unicos)]))

# Verificando alguns registros
df_relatos.iloc[:5, -1]

### Remoção de stopwords novamente

In [ ]:
# Aplicando stopwords à coluna relato_limpo
df_relatos['relato_limpo'] = df_relatos['relato_limpo'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# Verificando alguns exemplos
df_relatos.iloc[:5, -1]

### Remoção de excesso de espaços em branco

In [ ]:
# Removendo excesso de espaços em branco 
df_relatos['relato_limpo'] = df_relatos['relato_limpo'].str.replace('\s{2,100}', ' ', regex=True)
df_relatos.iloc[:5, -1]

### Criando Wordclouds

In [ ]:
# Verificando recurso para criação de imagem
from PIL import features
features.check('freetype2')

#### Wordcloud Reclamação

In [ ]:
# Criando objeto com textos das manifestações
texto_reclamacao = " ".join(texto for texto in df_relatos[df_relatos["tipoDeManifestacao"] == 'RECLAMACAO'].relato_limpo.astype(str))

# Mostrando quantidade de palavras o objeto
print ("Há {} palavras em manifestações de RECLAMACAO.".format(len(texto_reclamacao)))

In [ ]:
# Verificando 100 palavras mais frequentes
from collections import Counter
Counter(" ".join(df_relatos.loc[df_relatos["tipoDeManifestacao"] == 'RECLAMACAO', 'relato_limpo']).split()).most_common(100)

In [ ]:
# Gerando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate(texto_reclamacao)

# Mostrando a nuvem de palavras
plt.figure(figsize=(40,20))
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Wordcloud Dúvida

In [ ]:
# Criando objeto com textos das manifestações
texto_duvida = " ".join(texto for texto in df_relatos[df_relatos["tipoDeManifestacao"] == 'DUVIDA'].relato_limpo.astype(str))

# Mostrando quantidade de palavras o objeto
print ("Há {} palavras em manifestações de DUVIDA.".format(len(texto_duvida)))

In [ ]:
# Verificando 100 palavras mais frequentes
from collections import Counter
Counter(" ".join(df_relatos.loc[df_relatos["tipoDeManifestacao"] == 'DUVIDA', 'relato_limpo']).split()).most_common(100)

In [ ]:
# Gerando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate(texto_duvida)

# Mostrando a nuvem de palavras
plt.figure(figsize=(40,20))
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Wordcloud Elogio

In [ ]:
# Criando objeto com textos das manifestações
texto_elogio = " ".join(texto for texto in df_relatos[df_relatos["tipoDeManifestacao"] == 'ELOGIO'].relato_limpo.astype(str))

# Mostrando quantidade de palavras do objeto
print ("Há {} palavras em manifestações de ELOGIO.".format(len(texto_elogio)))

In [ ]:
# Verificando 100 palavras mais frequentes
from collections import Counter
Counter(" ".join(df_relatos.loc[df_relatos["tipoDeManifestacao"] == 'ELOGIO', 'relato_limpo']).split()).most_common(100)

In [ ]:
# Gerando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate(texto_elogio)

# Mostrando a nuvem de palavras
plt.figure(figsize=(40,20))
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Wordcloud Externa

In [ ]:
# Criando objeto com textos das manifestações
texto_externa = " ".join(texto for texto in df_relatos[df_relatos["tipoDeManifestacao"] == 'EXTERNA'].relato_limpo.astype(str))

# Mostrando quantidade de palavras do objeto
print ("Há {} palavras em manifestações de EXTERNA.".format(len(texto_externa)))

In [ ]:
# Verificando 100 palavras mais frequentes
from collections import Counter
Counter(" ".join(df_relatos.loc[df_relatos["tipoDeManifestacao"] == 'EXTERNA', 'relato_limpo']).split()).most_common(100)

In [ ]:
# Gerando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate(texto_externa)

# Mostrando a nuvem de palavras
plt.figure(figsize=(40,20))
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Wordcloud Outros

In [ ]:
# Criando objeto com textos das manifestações
texto_outros = " ".join(texto for texto in df_relatos[df_relatos["tipoDeManifestacao"] == 'OUTROS'].relato_limpo.astype(str))

# Mostrando quantidade de palavras do objeto
print ("Há {} palavras em manifestações de OUTROS.".format(len(texto_outros)))

In [ ]:
# Verificando 100 palavras mais frequentes
from collections import Counter
Counter(" ".join(df_relatos.loc[df_relatos["tipoDeManifestacao"] == 'OUTROS', 'relato_limpo']).split()).most_common(100)

In [ ]:
# Gerando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate(texto_outros)

# Mostrando a nuvem de palavras
plt.figure(figsize=(40,20))
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Wordcloud Sugestão

In [ ]:
# Criando objeto com textos das manifestações
texto_sugestao = " ".join(texto for texto in df_relatos[df_relatos["tipoDeManifestacao"] == 'SUGESTAO'].relato_limpo.astype(str))

# Mostrando quantidade de palavras do objeto
print ("Há {} palavras em manifestações de SUGESTAO.".format(len(texto_sugestao)))

In [ ]:
# Verificando 100 palavras mais frequentes
from collections import Counter
Counter(" ".join(df_relatos.loc[df_relatos["tipoDeManifestacao"] == 'SUGESTAO', 'relato_limpo']).split()).most_common(100)

In [ ]:
# Gerando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate(texto_sugestao)

# Mostrando a nuvem de palavras
plt.figure(figsize=(40,20))
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Wordcloud Acesso à Informação

In [ ]:
# Criando objeto com textos das manifestações
texto_acessoinformacao = " ".join(texto for texto in df_relatos[df_relatos["tipoDeManifestacao"] == 'ACESSOINFORMACAO'].relato_limpo.astype(str))

# Mostrando quantidade de palavras do objeto
print ("Há {} palavras em manifestações de ACESSOINFORMACAO.".format(len(texto_acessoinformacao)))

In [ ]:
# Verificando 100 palavras mais frequentes
from collections import Counter
Counter(" ".join(df_relatos.loc[df_relatos["tipoDeManifestacao"] == 'ACESSOINFORMACAO', 'relato_limpo']).split()).most_common(100)

In [ ]:
# Gerando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate(texto_acessoinformacao)

# Mostrando a nuvem de palavras
plt.figure(figsize=(40,20))
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Wordcloud Denúncia

In [ ]:
# Criando objeto com textos das manifestações
texto_denuncia = " ".join(texto for texto in df_relatos[df_relatos["tipoDeManifestacao"] == 'DENUNCIA'].relato_limpo.astype(str))

# Mostrando quantidade de palavras do objeto
print ("Há {} palavras em manifestações de DENUNCIA.".format(len(texto_denuncia)))

In [ ]:
# Verificando 100 palavras mais frequentes
from collections import Counter
Counter(" ".join(df_relatos.loc[df_relatos["tipoDeManifestacao"] == 'DENUNCIA', 'relato_limpo']).split()).most_common(100)

In [ ]:
# Gerando a nuvem de palavras
wordcloud = WordCloud(background_color="white", width=800, height=400).generate(texto_denuncia)

# Mostrando a nuvem de palavras
plt.figure(figsize=(40,20))
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Stemming

In [ ]:
# Importando pacote
import nltk
nltk.download('rslp')

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=128)
def stemming_portugues(texto, stemmer):
    lista_palavras = texto.split()
    lista_stems = []
    
    for palavra in lista_palavras:
        lista_stems.append(stemmer.stem(palavra))
    
    return ' '.join(lista_stems)

In [ ]:
stemmer = nltk.stem.RSLPStemmer()

df_relatos['relato_stemm'] = df_relatos['relato_limpo'].apply(lambda palavra: stemming_portugues(palavra, stemmer))

df_relatos.iloc[:5, -1]

In [ ]:
df_relatos.iloc[:5, 8:11]

# Ajuste de base de dados para fastText

In [ ]:
# Criando coluna com label e relato juntos para treinamento no fastText
df_relatos['label_texto'] = '__label__' + df_relatos['tipoDeManifestacao'] + ' ' + df_relatos['relato_stemm']

In [ ]:
df_relatos['label_texto'].tail(20)

In [ ]:
# Criando dataframe com label e texto juntos
df_fasttext = df_relatos['label_texto']

In [ ]:
# Salvando arquivo txt com label e texto juntos
df_fasttext.to_csv('data/processed/label_texto.txt', index=False)

# Treinamento

In [ ]:
# Importanto biblioteca
import fasttext

In [ ]:
# Realizando a divisão de dados entre treino (80%) e teste (20%)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_relatos['label_texto'], df_relatos['tipoDeManifestacao'], test_size=0.2, stratify=df_relatos['tipoDeManifestacao'].values)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Verificando dados de treino
X_train.head()

## Modelo simples

In [ ]:
# Importando biblioteca para realizar divisão estratificada de folds 
from sklearn.model_selection import StratifiedKFold

# Criando função para realizar a validação cruzada
def validacao_cruzada(X, y, parametros_modelo=None):
    """
    Função para rodar validação cruzada com kfold em estratos (amostragem extratificada)
    """
    
    resultados = []
    
    # 5 splits
    skfold = StratifiedKFold(n_splits=5)
    
    # para cada, uma iteração com os índices
    for train_index, test_index in skfold.split(X, y):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]

        # não utilizou-se o y porque o label já está no x
        # y_train, y_test = y[train_index], y[test_index]
        
        # o fasttext só aceita input de arquivos, então temos que salvar
        X_train.to_csv('data/train/manifestacao.train', header=False, index=False)
        X_test.to_csv('data/train/manifestacao.valid', header=False, index=False)
    
        # se foram passados parâmetros, mandar kwargs para o treinamento
        if parametros_modelo:
            modelo = fasttext.train_supervised(input="data/train/manifestacao.train", **parametros_modelo)    
        else:
            modelo = fasttext.train_supervised(input="data/train/manifestacao.train")
            
        resultado = modelo.test("data/train/manifestacao.valid")
        print(f'Resultado: {resultado}')
        resultados.append(resultado[1])
    
    return resultados

In [ ]:
type(X_train)

In [ ]:
X_test.str.extract("__label__(.*?) ").value_counts()

## Treinamentos com diferentes hiperparâmetros

### Treinamento 1 (padrão)

In [ ]:
%%time

# Precisamos passar como dataframe
resultado = validacao_cruzada(X_train.to_frame(), y_train)
resultado

In [ ]:
np.array(resultado).mean()

### Treinamento 2

In [ ]:
%%time

# Tunagem de hiperparametros, aumentando ngrams para 3 e épocas para 20
hyper_params = {"epoch": 20,
    "wordNgrams": 3,
    "dim": 100
}

resultado = validacao_cruzada(X_train.to_frame(), y_train, hyper_params)
resultado

In [ ]:
np.array(resultado).mean()

### Treinamento 3

In [ ]:
%%time

# Tunagem de hiperparametros, aumentando ngrams para 3 e épocas para 50

hyper_params = {"epoch": 50,
    "wordNgrams": 3,
    "dim": 100
}

resultado = validacao_cruzada(X_train.to_frame(), y_train, hyper_params)
resultado

In [ ]:
np.array(resultado).mean()

### Treinamento 4

In [ ]:
%%time
# Tunagem de hiperparametros, aumentando ngrams para 4 e épocas para 50

hyper_params = {"epoch": 50,
    "wordNgrams": 4,
    "dim": 100
}

resultado = validacao_cruzada(X_train.to_frame(), y_train, hyper_params)
resultado

In [ ]:
np.array(resultado).mean()

### Treinamento 5

In [ ]:
%%time
# Tunagem de hiperparametros, aumentando dimensao dos embeddings para 200, ngrams para 3 e epocas para 20

hyper_params = {"epoch": 20,
    "wordNgrams": 3,
    "dim": 200
}

resultado = validacao_cruzada(X_train.to_frame(), y_train, hyper_params)
resultado

In [ ]:
np.array(resultado).mean()

### Treinamento 6

In [ ]:
%%time
# Tunagem de hiperparametros, aumentando ngrams para 3 e épocas para 20

hyper_params = {"epoch": 20,
    "wordNgrams": 3,
    "dim": 100,
    "lr": 0.01
}

resultado = validacao_cruzada(X_train.to_frame(), y_train, hyper_params)
resultado

In [ ]:
np.array(resultado).mean()

### Treinamento 7

In [ ]:
%%time
# Tunagem de hiperparametros, aumentando ngrams para 3 e épocas para 20

hyper_params = {"epoch": 50,
    "wordNgrams": 3,
    "dim": 100,
    "lr": 0.01
}

resultado = validacao_cruzada(X_train.to_frame(), y_train, hyper_params)

resultado

In [ ]:
np.array(resultado).mean()

### Treinamento 8

In [ ]:
%%time
# Tunagem de hiperparametros

hyper_params = {"epoch": 20,
    "wordNgrams": 3,
    "dim": 100,
    "lr": 0.1
}

resultado = validacao_cruzada(X_train.to_frame(), y_train, hyper_params)
resultado

In [ ]:
np.array(resultado).mean()

### Treinamento 9

In [ ]:
%%time
# Tunagem de hiperparametros

hyper_params = {"epoch": 20,
    "wordNgrams": 3,
    "dim": 100,
    "lr": 0.2
}

resultado = validacao_cruzada(X_train.to_frame(), y_train, hyper_params)
resultado

In [ ]:
np.array(resultado).mean()

### Treinamento 10

In [ ]:
%%time
# Tunagem de hiperparametros

hyper_params = {"epoch": 20,
    "wordNgrams": 3,
    "dim": 50,
    "lr": 0.2
}

resultado = validacao_cruzada(X_train.to_frame(), y_train, hyper_params)
resultado

In [ ]:
np.array(resultado).mean()

# Modelo final

A partir do modelo final, escolhido com base nos valores de cross validation, treinamos com a base de treino toda.

In [ ]:
# Definindo base de treino e base de teste
X_train.to_csv('data/train/manifestacao.train', header=False, index=False)
X_test.to_csv('data/train/manifestacao.test', header=False, index=False)

In [ ]:
# Definindo hiperparâmetros para treino do modelo final
melhores_hiperparametros = {"epoch": 20,
    "wordNgrams": 3,
    "dim": 50,
    "lr": 0.2
}

# Treinando modelo final
model_final = fasttext.train_supervised(input="data/train/manifestacao.train", **melhores_hiperparametros)

In [ ]:
# Salvando o modelo final na pasta model
model_final.save_model("model/model_tun.bin")

In [ ]:
# Verificando precisão do modelo final
model_final.test("data/train/manifestacao.test")

In [ ]:
# Verificando precisão do modelo final
model_final.test_label("data/train/manifestacao.test")

# Modelo de produção

Como o resultado no teste foi satisfatório, podemos retreinar o modelo com os melhores hiperparâmetros com a base completa

In [ ]:
# Contatenando dados de treino e teste para obter base completa
base_completa = pd.concat([X_train, X_test], axis=0)
base_completa.shape

In [ ]:
# Criando arquivo CSV com base completa
base_completa.to_csv('data/train/manifestacao.train', header=False, index=False)

In [ ]:
# Treinando modelo de produção
modelo_producao = fasttext.train_supervised(input="data/train/manifestacao.train", **melhores_hiperparametros)

In [ ]:
# Salvando o modelo de produção
modelo_producao.save_model("models/production_model.bin")